Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [305]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [306]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [307]:
def available_actions(state: np.ndarray) -> list[action]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [308]:

import numpy as np
from collections import namedtuple
from heapq import heappop, heappush
from random import choice
from tqdm import tqdm

Various heuristics


In [309]:
# Manhattan distance heuristic
def manhattan(state: np.ndarray, goal: np.ndarray) -> int:
    distance = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = state[i, j]
            if value != 0:
                distances = np.where(goal == value)
                distance += abs(i - distances[0][0]) + abs(j - distances[1][0])
    return distance

def hamming(state: np.ndarray, goal: np.ndarray) -> int:
    return np.sum(state != goal) - 1  # Exclude the empty tile

import numpy as np

def linear_conflict_manhattan(state: np.ndarray, goal: np.ndarray) -> int:
    def linear_conflict(state, goal):
        """Compute linear conflicts."""
        conflict_count = 0
        # Check rows
        for row in range(PUZZLE_DIM):
            tiles_in_row = [state[row, col] for col in range(PUZZLE_DIM) if state[row, col] != 0]
            goal_positions = {}
            for tile in tiles_in_row:
                distances = np.where(goal == tile)
                goal_positions[tile] = distances[0][0], distances[1][0]
            for i, tile1 in enumerate(tiles_in_row):
                for tile2 in tiles_in_row[i + 1:]:
                    if goal_positions[tile1] > goal_positions[tile2]:
                        conflict_count += 1
        # Check columns
        for col in range(PUZZLE_DIM):
            tiles_in_col = [state[row, col] for row in range(PUZZLE_DIM) if state[row, col] != 0]
            goal_positions = {}
            for tile in tiles_in_col:
                distances = np.where(goal == tile)
                goal_positions[tile] = distances[0][0], distances[1][0]
            for i, tile1 in enumerate(tiles_in_col):
                for tile2 in tiles_in_col[i + 1:]:
                    if goal_positions[tile1] > goal_positions[tile2]:
                        conflict_count += 1
        return 2 * conflict_count  # Each conflict adds 2 moves

    # Combine Manhattan distance and linear conflict
    return manhattan(state, goal) + linear_conflict(state, goal)


Algos

In [310]:
# A*  or Dijkstra's search

def search_astar(initial_state: np.ndarray, goal_state: np.ndarray, heuristic, use_heuristic: bool = True):
    """
    Unified search algorithm for A* and Dijkstra's based on the use_heuristic flag.
    
    Args:
        initial_state (np.ndarray): Initial puzzle state.
        goal_state (np.ndarray): Goal puzzle state.
        use_heuristic (bool): If True, uses A*; if False, uses Dijkstra's algorithm.
    
    Returns:
        list: Sequence of actions leading to the goal state, or an error message if unsolvable.
    """

    visited = set()
    priority_queue = []
    heappush(priority_queue, (0, 0, initial_state.tobytes(), []))  # (priority, cost, state, path)
    steps = 0

    while priority_queue:
        _, cost, current_state_bytes, path = heappop(priority_queue)
        current_state = np.frombuffer(current_state_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)

        if np.array_equal(current_state, goal_state):
            #print cost and steps
            print(f"Steps: {steps}, Cost: {cost}")
            return path

        visited.add(current_state_bytes)

        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_state_bytes = new_state.tobytes()
            if new_state_bytes not in visited:
                new_cost = cost + 1
                # Priority is cost + heuristic for A*, or just cost for Dijkstra's
                priority = new_cost + (heuristic(new_state, goal_state) if use_heuristic else 0)
                heappush(priority_queue, (priority, new_cost, new_state_bytes, path + [act]))

        steps += 1
        if steps % 100_000 == 0:
            print(f"Steps: {steps}, Cost: {cost}")
        if steps > 1_000_000:
            #Print solution and return
            print("Final state: \n", current_state)
            print(f"Steps: {steps}, Cost: {cost}")
            return path

    return "No solution found."


## Create solvable puzzle


In [311]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
RANDOMIZE_STEPS = 100_000
state = goal_state.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))


print("Solvable puzzle generated.")


Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 52217.14it/s]

Solvable puzzle generated.


In [312]:
# Which algorithm to run?
RUN = 'A*'
#RUN = 'Dijkstra'  
# Solve the puzzle
switch = {
    'A*': search_astar,
    'Dijkstra': search_astar
}
use_heuristic = True
if RUN == 'Dijkstra':
    use_heuristic = False

HEURISTIC = 'linear_conflict_manhattan'  # 'manhattan', 'hamming', 'linear_conflict_manhattan'
switch_heuristic = {
    'manhattan': manhattan,
    'hamming': hamming,
    'linear_conflict_manhattan': linear_conflict_manhattan
}
heuristic = switch_heuristic[HEURISTIC]

# Test1 for state 2 8 7 3 6 5 0 1 4
#state = np.array([[2, 8, 7], [3, 6, 5], [0, 1, 4]])
# Test2 for state 8 6 0 1 7 2 4 5 3
#state = np.array([[8, 6, 0], [1, 7, 2], [4, 5, 3]])

#4x4 puzzle
# Test1 for state 10 1 13 6 8 15 4 12 0 9 2 5 3 14 11 7
state = np.array([[10, 1, 13, 6], [8, 15, 4, 12], [0, 9, 2, 5], [3, 14, 11, 7]])
# Test2 for state 11 6 14 13 4 3 15 2 9 10 0 5 8 1 7 12
#state = np.array([[11, 6, 14, 13], [4, 3, 15, 2], [9, 10, 0, 5], [8, 1, 7, 12]])
# Very long test

# Test3 for state 8 2 0 14 10 13 11 1 9 12 15 7 6 5 4 3
#state = np.array([[8, 2, 0, 14], [10, 13, 11, 1], [9, 12, 15, 7], [6, 5, 4, 3]])


print("Initial state:")
print(state)


solve_puzzle = switch[RUN]
solution = solve_puzzle(state, goal_state, heuristic, use_heuristic)

print("Checking solution...")
# Check if the solution is valid
if isinstance(solution, list):
    state = state.copy()
    for act in solution:
        state = do_action(state, act)
    if not np.array_equal(state, goal_state):
        solution = "Invalid solution."

print("\nSolution steps:")
if isinstance(solution, list):
    for act in solution:
        print(act.pos1, '->', act.pos2)
else:
    print(solution)

Initial state:
[[10  1 13  6]
 [ 8 15  4 12]
 [ 0  9  2  5]
 [ 3 14 11  7]]
Steps: 13298, Cost: 64
Checking solution...

Solution steps:
(2, 0) -> (2, 1)
(2, 1) -> (2, 2)
(2, 2) -> (2, 3)
(2, 3) -> (1, 3)
(1, 3) -> (1, 2)
(1, 2) -> (1, 1)
(1, 1) -> (1, 0)
(1, 0) -> (0, 0)
(0, 0) -> (0, 1)
(0, 1) -> (1, 1)
(1, 1) -> (2, 1)
(2, 1) -> (2, 2)
(2, 2) -> (1, 2)
(1, 2) -> (0, 2)
(0, 2) -> (0, 1)
(0, 1) -> (1, 1)
(1, 1) -> (2, 1)
(2, 1) -> (2, 0)
(2, 0) -> (3, 0)
(3, 0) -> (3, 1)
(3, 1) -> (3, 2)
(3, 2) -> (2, 2)
(2, 2) -> (1, 2)
(1, 2) -> (0, 2)
(0, 2) -> (0, 3)
(0, 3) -> (1, 3)
(1, 3) -> (2, 3)
(2, 3) -> (3, 3)
(3, 3) -> (3, 2)
(3, 2) -> (2, 2)
(2, 2) -> (2, 3)
(2, 3) -> (1, 3)
(1, 3) -> (1, 2)
(1, 2) -> (1, 1)
(1, 1) -> (1, 0)
(1, 0) -> (2, 0)
(2, 0) -> (2, 1)
(2, 1) -> (1, 1)
(1, 1) -> (1, 0)
(1, 0) -> (0, 0)
(0, 0) -> (0, 1)
(0, 1) -> (1, 1)
(1, 1) -> (1, 2)
(1, 2) -> (0, 2)
(0, 2) -> (0, 1)
(0, 1) -> (0, 0)
(0, 0) -> (1, 0)
(1, 0) -> (1, 1)
(1, 1) -> (1, 2)
(1, 2) -> (2, 2)
(2, 2) -> (3,